<a href="https://colab.research.google.com/github/space4VV/LLM_trailblazr/blob/main/chapter2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
import urllib.request
if not os.path.exists("the_verdict.txt"):
  url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
  file_path = "the_verdict.txt"

  urllib.request.urlretrieve(url,file_path)


In [5]:
with open("the_verdict.txt","r",encoding="utf-8") as f:
  r_text = f.read()

In [7]:
r_text[0:500]

'I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no great surprise to me to hear that, in the height of his glory, he had dropped his painting, married a rich widow, and established himself in a villa on the Riviera. (Though I rather thought it would have been Rome or Florence.)\n\n"The height of his glory"--that was what the women called it. I can hear Mrs. Gideon Thwing--his last Chicago sitter--deploring his unaccountable abdication. "Of course it\''

In [9]:
#total no:characters
len(r_text)

20479

## Simple tokenizer

In [13]:
import re
# some random text
text = "Hello i am vishnu, and i am learning LLM!"
result = re.split(r'\s',text)
print(result)
# the pblm is here we are not splitting the punctuations

['Hello', 'i', 'am', 'vishnu,', 'and', 'i', 'am', 'learning', 'LLM!']


In [14]:
result = re.split(r'([,.]|\s)', text) ()
print(result)
# we have empty strings here

['Hello', ' ', 'i', ' ', 'am', ' ', 'vishnu', ',', '', ' ', 'and', ' ', 'i', ' ', 'am', ' ', 'learning', ' ', 'LLM!']


In [15]:
result= [tok for tok in result if tok.strip()]
print(result)

['Hello', 'i', 'am', 'vishnu', ',', 'and', 'i', 'am', 'learning', 'LLM!']


In [17]:
# nice lets add other punctuations too
def simple_tokenizer(text):
  result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
  result = [tok for tok in result if tok.strip()]
  return result

result = simple_tokenizer(text)
print(result)

['Hello', 'i', 'am', 'vishnu', ',', 'and', 'i', 'am', 'learning', 'LLM', '!']


In [18]:
# lets do with the raw text
preprocessed = simple_tokenizer(r_text)
print(preprocessed[0:100])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in', 'the', 'height', 'of', 'his', 'glory', ',', 'he', 'had', 'dropped', 'his', 'painting', ',', 'married', 'a', 'rich', 'widow', ',', 'and', 'established', 'himself', 'in', 'a', 'villa', 'on', 'the', 'Riviera', '.', '(', 'Though', 'I', 'rather', 'thought', 'it', 'would', 'have', 'been', 'Rome', 'or', 'Florence', '.', ')', '"', 'The', 'height', 'of', 'his', 'glory', '"', '--', 'that', 'was', 'what', 'the', 'women', 'called', 'it', '.', 'I', 'can', 'hear', 'Mrs', '.', 'Gideon', 'Thwing', '--', 'his', 'last', 'Chicago', 'sitter', '--']


In [19]:
print(len(preprocessed))

4690
